In [1]:
from pyspark.sql import SparkSession
import pyspark as ps
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, when

In [2]:
#abrindo sessão
spark=(SparkSession
       .builder
       .appName("Python Spark SQL basic example") 
        .config("spark.some.config.option", "some-value") 
        .getOrCreate())
conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")

In [3]:
#Exercício 1
casas = (spark.read.format("csv")
      .option("header","true")
      .load("realtor-data.zip.csv"))
casas.show(10)

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|   00601|     920.0|          NULL|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|   00601|    1527.0|          NULL|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|   00795|     748.0|          NULL|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|   00731|    1800.0|          NULL|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|   00680|      NULL|          NULL|
|   103378.0|for_sale|179000.0| 

In [4]:
#Exercício 2
casas_pd = casas.toPandas()
len(casas_pd)


2226382

In [5]:
casas.count()

2226382

In [6]:
#Exercício 3
casas.printSchema()

root
 |-- brokered_by: string (nullable = true)
 |-- status: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- bath: string (nullable = true)
 |-- acre_lot: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- house_size: string (nullable = true)
 |-- prev_sold_date: string (nullable = true)



In [8]:
#filtro
casas.filter((col('state')=="Puerto Rico")&(col('price')>100000)).show(10)

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|   00601|     920.0|          NULL|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|   00731|    1800.0|          NULL|
|   103378.0|for_sale|179000.0|  4|   3|    0.46|1850806.0|San Sebastian|Puerto Rico|   00612|    2520.0|          NULL|
|    65672.0|for_sale|300000.0|  5|   3|    7.46|1946226.0|   Las Marias|Puerto Rico|   00670|    5403.0|          NULL|
|    52707.0|for_sale|150000.0|  3|   2|    0.08|1773902.0|   Juana Diaz|Puerto Rico|   00795|    1045.0|          NULL|
|    46019.0|for_sale|155000.0| 

In [9]:
casas.withColumn("preço", casas["price"].cast("float")).filter(
    (col('state')=="Puerto Rico")&(col('preço')>100000)).show(10)

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+--------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|   preço|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+--------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|   00601|     920.0|          NULL|105000.0|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|   00731|    1800.0|          NULL|145000.0|
|   103378.0|for_sale|179000.0|  4|   3|    0.46|1850806.0|San Sebastian|Puerto Rico|   00612|    2520.0|          NULL|179000.0|
|    65672.0|for_sale|300000.0|  5|   3|    7.46|1946226.0|   Las Marias|Puerto Rico|   00670|    5403.0|          NULL|300000.0|
|    52707.0|for_sale|150000.0|  3|   2|    0.08|1773902.0|   Juana Diaz|Puerto Rico|   00

In [11]:
#Exercício 4
casas.withColumn('tamanho',
                when(col('house_size')<1000,'pequena')
                .when(col('house_size')<5000,'media')
                .when(col('house_size')>=5000,'grande')
                .otherwise('sem info')).show(20)

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+--------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date| tamanho|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+--------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|   00601|     920.0|          NULL| pequena|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|   00601|    1527.0|          NULL|   media|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|   00795|     748.0|          NULL| pequena|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|   00731|    1800.0|          NULL|   media|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|   00

In [12]:
spark.stop()